# Building graphs with Metric Learning (and an introduction to the new pytorch lightning-based framework)

In [4]:
from functools import partial

import torch

from gnn_tracking.training.ml import MLModule
from gnn_tracking.models.graph_construction import GraphConstructionFCNN
from gnn_tracking.metrics.losses import GraphConstructionHingeEmbeddingLoss
from pytorch_lightning import Trainer
from gnn_tracking.utils.loading import TrackingDataModule

from torch_geometric.data import Data
from torch import nn
from pytorch_lightning.core.mixins import HyperparametersMixin

## Step 1: Configuring the data

The configuration for train/val/test data and its dataloader is held in the `TrackingDataModule` (subclass of `LightningDataModule`).

In [5]:
dm = TrackingDataModule(
    train=dict(
        dirs=["/Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/"],
        stop=5,
    ),
    val=dict(
        dirs=["/Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/"],
        start=5,
        stop=10,
    ),
    # could also configure a 'test' set here
)

Other keys allow to configure the loaders (batch size, number of workers, etc.). See the docstring of `TrackingDataModule` for details.

### Details (for understanding)

Note that all of the following will be done implicitly by the `Trainer` and you won't have to worry about it. But if you want to inspect the data, you can do so.

When calling the `setup` method, the `LightningDataModule` initializes instances of `TrackingDataset` (`torch_geometric.Dataset`) for each of these. We can get the corresponding dataloaders by calling `dm.train_dataloader()` and analog for validation and test.

Example:

In [6]:
# This is called by the Trainer automatically and sets up the datasets
dm.setup(stage="fit")  # 'fit' combines 'train' and 'val'
# Now the datasets are available:
dm.datasets

[13:00:44] INFO: DataLoader will load 5 graphs (out of 90 available).
[13:00:44] DEBUG: First graph is /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/data21025_s0.pt, last graph is /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/data21053_s0.pt
[13:00:44] INFO: DataLoader will load 5 graphs (out of 90 available).
[13:00:44] DEBUG: First graph is /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/data21058_s0.pt, last graph is /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/data21094_s0.pt


{'train': TrackingDataset(5), 'val': TrackingDataset(5)}

For example, we can inspect the first element of the training dataset:

In [7]:
data = dm.datasets["train"][0]

To get the corresponding dataloaders, use one of the methods (but again, you probalby won't need to):

In [7]:
dm.train_dataloader(), dm.val_dataloader()

(<torch_geometric.loader.dataloader.DataLoader at 0x147d2f090>,
 <torch_geometric.loader.dataloader.DataLoader at 0x15815d950>)

## Step 2: Configuring a model

We write a normal `torch.nn.Module`. The easiest way is to import one of the modules that we have already written in the `gnn_tracking` librar.

In [25]:
model = GraphConstructionFCNN(in_dim=14, out_dim=8, depth=5, hidden_dim=64)

However, you can also write your own. Here is a very simple one:

In [15]:
class DemoGraphConstructionModel(nn.Module, HyperparametersMixin):
    def __init__(
        self,
        in_dim: int,
        hidden_dim: int,
        out_dim: int,
        depth: int = 5,
    ):
        super().__init__()
        # This is made available by the HyperparametersMixin
        # all of our hyperparameters from the __init__ arguments
        # are saved to self.hparams (but we don't need this in this
        # example)
        self.save_hyperparameters()
        assert depth > 2
        _layers = [
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(),
        ]
        for _ in range(depth - 2):
            _layers.append(nn.Linear(hidden_dim, hidden_dim))
            _layers.append(nn.ReLU())
        _layers.append(nn.Linear(hidden_dim, out_dim))
        self._model = nn.Sequential(*_layers)

    def forward(self, data: Data):
        # Our trainer class will expect us to return a dictionary, where
        # the key H has the transformed latent space.
        return {"H": self._model(data.x)}

In [26]:
# model = DemoGraphConstructionModel(in_dim=14, out_dim=8, hidden_dim=64)

If you are familiar with normal pytorch, there was only few differences:

1. We inherit from `HyperparamsMixin`
2. We call `self.save_hyperparameters()`

### Details (for understanding)

We saved all hyperparameters:

In [27]:
model.hparams

"beta":       0.4
"depth":      5
"hidden_dim": 64
"in_dim":     14
"out_dim":    8

Note how `depth=5` was saved despite not being specified explicitly (it was recognized as a default parameter).

As always, you can simply evaluate the `model` on a piece of data:

In [28]:
out = model(data)

## Step 3: Configuring loss functions, metrics and the lightning module

The pytorch model is bundled together with a set of loss functions (just one here), that we backpropagate from in the training step, and a set of metrics. Together, these components make up the `LightningModule` that we pass to the pytorch lightning `Trainer` for training.

If you were familiar with our previous `TCNTrainer` training class, this `MLModule` now fulfills (almost) the exact same role.

In [19]:
lmodel = MLModule(
    model=model,
    loss_fct=GraphConstructionHingeEmbeddingLoss(max_num_neighbors=10),
    lw_repulsive=0.5,  # loss weight, see below
    optimizer=partial(torch.optim.Adam, lr=1e-4),
)

[13:03:20] DEBUG: Got obj of type <class '__main__.DemoGraphConstructionModel'>, assuming I have to save hyperparameters
[13:03:20] DEBUG: Saving hyperperameters {'class_path': '__main__.DemoGraphConstructionModel', 'init_args': {'in_dim': 14, 'hidden_dim': 64, 'out_dim': 8, 'depth': 5}}
[13:03:20] DEBUG: Got obj of type <class 'gnn_tracking.metrics.losses.GraphConstructionHingeEmbeddingLoss'>, assuming I have to save hyperparameters
[13:03:20] DEBUG: Saving hyperperameters {'class_path': 'gnn_tracking.metrics.losses.GraphConstructionHingeEmbeddingLoss', 'init_args': {'r_emb': 0.002, 'max_num_neighbors': 10, 'attr_pt_thld': 0.9, 'p_attr': 1, 'p_rep': 1}}


### Details (for understanding)

Again, all hyperparameters are accessible (even the ones that weren't explicitly specified but only set by default):

In [20]:
lmodel.hparams

"loss_fct":     {'class_path': 'gnn_tracking.metrics.losses.GraphConstructionHingeEmbeddingLoss', 'init_args': {'r_emb': 0.002, 'max_num_neighbors': 10, 'attr_pt_thld': 0.9, 'p_attr': 1, 'p_rep': 1}}
"lw_repulsive": 0.5
"model":        {'class_path': '__main__.DemoGraphConstructionModel', 'init_args': {'in_dim': 14, 'hidden_dim': 64, 'out_dim': 8, 'depth': 5}}

As you can see, any _objects_ that were passed to the model are also saved to the hyperparameters in a way that we can bring them back.

The loss function takes output from the model and the data and returns two separate losses:

In [21]:
loss_fct = GraphConstructionHingeEmbeddingLoss()
loss_fct(
    x=out["H"],
    particle_id=data.particle_id,
    batch=data.batch,
    edge_index=data.edge_index,
    pt=data.pt,
)

{'attractive': tensor(2.0991, grad_fn=<DivBackward0>),
 'repulsive': tensor(2.5671e-06, grad_fn=<SumBackward0>)}

Both parts of the loss functions are combined with the loss weight we have configured above (weight of 1 for attractive, weight of 0.5 for repulsive). All of this is done in `MLModule.get_losses` (returning the total loss and a dictionary of the individual losses):

In [22]:
lmodel.get_losses(out, data)

(tensor(2.0991, grad_fn=<AddBackward0>),
 {'attractive': 2.09908127784729,
  'repulsive': 2.5670772174635204e-06,
  'attractive_weighted': 2.09908127784729,
  'repulsive_weighted': 1.2835386087317602e-06})

## Step 4: Training

In [23]:
trainer = Trainer(max_epochs=1, accelerator="cpu", log_every_n_steps=1)
trainer.fit(model=lmodel, datamodule=dm)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/fuchur/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(
[13:08:53] INFO: DataLoader will load 5 graphs (out of 90 available).
[13:08:53] DEBUG: First graph is /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/data21025_s0.pt, last graph is /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/data21053_s0.pt
[13:08:53] INFO: DataLoader will load 5 graphs (out of 90 available).
[13:08:53] DEBUG: First graph is /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/data21058_s0.pt, last graph is /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/data21094_s0.pt

  | Name     | Type                                | Params
----------------------------------

Sanity Checking: 0it [00:00, ?it/s]

/Users/fuchur/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/fuchur/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 6789. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/Users/fuchur/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 8459. To avoid any miscalculations, use `self.log(..., batch_size=batch_siz

           Validation epoch=0            
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━┓
┃ Metric              ┃   Value ┃ Error ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━┩
│ attractive          │ 2.12333 │   nan │
│ attractive_weighted │ 2.12333 │   nan │
│ repulsive           │ 0.00000 │   nan │
│ repulsive_weighted  │ 0.00000 │   nan │
│ total               │ 2.12333 │   nan │
└─────────────────────┴─────────┴───────┘

/Users/fuchur/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/Users/fuchur/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 7299. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/Users/fuchur/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 5946. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/Users/fuchur/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 8207. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


           Validation epoch=0            
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━┓
┃ Metric              ┃   Value ┃ Error ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━┩
│ attractive          │ 1.94851 │   nan │
│ attractive_weighted │ 1.94851 │   nan │
│ repulsive           │ 0.00000 │   nan │
│ repulsive_weighted  │ 0.00000 │   nan │
│ total               │ 1.94851 │   nan │
└─────────────────────┴─────────┴───────┘

`Trainer.fit` stopped: `max_epochs=1` reached.


## Restoring a pre-trained model

Take a look at the `lightning_logs` directory:

In [24]:
! ls lightning_logs

version_0   version_118 version_22  version_42  version_62  version_82
version_1   version_119 version_23  version_43  version_63  version_83
version_10  version_12  version_24  version_44  version_64  version_84
version_100 version_120 version_25  version_45  version_65  version_85
version_101 version_121 version_26  version_46  version_66  version_86
version_102 version_122 version_27  version_47  version_67  version_87
version_103 version_123 version_28  version_48  version_68  version_88
version_104 version_124 version_29  version_49  version_69  version_89
version_105 version_125 version_3   version_5   version_7   version_9
version_106 version_126 version_30  version_50  version_70  version_90
version_107 version_127 version_31  version_51  version_71  version_91
version_108 version_128 version_32  version_52  version_72  version_92
version_109 version_13  version_33  version_53  version_73  version_93
version_11  version_14  version_34  version_54  version_74  version_94
version

In [40]:
! ls lightning_logs/version_0/checkpoints

epoch=0-step=5.ckpt


Navigate to one of the versions and take a look at the `hparams.yaml` file. It should contain exactly the hyperparameters from the run.


In [38]:
! cat lightning_logs/version_0/hparams.yaml

model:
  class_path: gnn_tracking.models.graph_construction.GraphConstructionFCNN
  init_args:
    in_dim: 14
    hidden_dim: 64
    out_dim: 8
    depth: 5
    beta: 0.4
lw_repulsive: 0.5
loss_fct:
  class_path: gnn_tracking.metrics.losses.GraphConstructionHingeEmbeddingLoss
  init_args:
    r_emb: 0.002
    max_num_neighbors: 10
    attr_pt_thld: 0.9
    p_attr: 1
    p_rep: 1


We can bring back the trained model by loading one of the checkpoints:

In [42]:
restored_model = MLModule.load_from_checkpoint(
    "lightning_logs/version_0/checkpoints/epoch=0-step=5.ckpt"
)

[19:39:17] DEBUG: Got {'class_path': 'gnn_tracking.models.graph_construction.GraphConstructionFCNN', 'init_args': {'in_dim': 14, 'hidden_dim': 64, 'out_dim': 8, 'depth': 5, 'beta': 0.4}}, assuming I have to load
[19:39:17] DEBUG: Getting class GraphConstructionFCNN from module gnn_tracking.models.graph_construction
[19:39:17] DEBUG: Got {'class_path': 'gnn_tracking.metrics.losses.GraphConstructionHingeEmbeddingLoss', 'init_args': {'r_emb': 0.002, 'max_num_neighbors': 10, 'attr_pt_thld': 0.9, 'p_attr': 1, 'p_rep': 1}}, assuming I have to load
[19:39:17] DEBUG: Getting class GraphConstructionHingeEmbeddingLoss from module gnn_tracking.metrics.losses


Note how we didn't have to specify any the hyperparameters again.

However, we can easily change some of them by adding them as additional keyword arguments.

In [45]:
restored_model_modified = MLModule.load_from_checkpoint(
    "lightning_logs/version_0/checkpoints/epoch=0-step=5.ckpt",
    lw_repulsive=0.1,
    loss_fct=GraphConstructionHingeEmbeddingLoss(max_num_neighbors=5),
)

[19:40:11] DEBUG: Got {'class_path': 'gnn_tracking.models.graph_construction.GraphConstructionFCNN', 'init_args': {'in_dim': 14, 'hidden_dim': 64, 'out_dim': 8, 'depth': 5, 'beta': 0.4}}, assuming I have to load
[19:40:11] DEBUG: Getting class GraphConstructionFCNN from module gnn_tracking.models.graph_construction
[19:40:11] DEBUG: Got obj of type <class 'gnn_tracking.metrics.losses.GraphConstructionHingeEmbeddingLoss'>, assuming I have to save hyperparameters
[19:40:11] DEBUG: Saving hyperperameters {'class_path': 'gnn_tracking.metrics.losses.GraphConstructionHingeEmbeddingLoss', 'init_args': {'r_emb': 0.002, 'max_num_neighbors': 5, 'attr_pt_thld': 0.9, 'p_attr': 1, 'p_rep': 1}}


Note that you cannot modify the model architecture however (but you could in principle change the `beta` parameter of the residual connections).

## Running all of this from the command line

All of the following can be achieved by running the following command:

```bash
python3 gnn_tracking/trainers/run.py fit --model configs/model.yml --data configs/data.yml  --trainer.accelerator cpu --trainer.accelerator cpu
```

with the data config file

```yaml
train:
  dirs:
    - /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/
  stop: 5
test:
  dirs:
    - /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/
  star: 10
  stop: 15
val:
  dirs:
    - /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/
  start: 5
  stop: 10
```

and model config file:

```
class_path: gnn_tracking.training.ml.MLModule
init_args:
  model:
    class_path: gnn_tracking.models.graph_construction.GraphConstructionFCNN
    init_args:
      in_dim: 14
      out_dim: 8
      hidden_dim: 512
      depth: 5
  lw_repulsive: 0.5
  loss_fct:
    class_path: gnn_tracking.metrics.losses.GraphConstructionHingeEmbeddingLoss
    init_args: {}
  optimizer:
    class_path: torch.optim.Adam
    init_args:
      lr: 0.0001
```

To quickly override one of the options, you can simply add them to the command line, e.g., `--model.init_args.lw_repulsive=0.1` or `--model.model.init_args.depth=6`.